In [1]:
import torch
from torch import nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)  # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        acc = accuracy(out, labels)  # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()  # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))


def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class Cifar10CnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: 256 x 4 x 4

            nn.Flatten(),
            nn.Linear(576, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 13))

    def forward(self, xb):
        return self.network(xb)


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        # Third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        # Fully connected layer
        self.fc1 = nn.Linear(128 * 25 * 25, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 13)  # 13 output classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply conv1 + relu + max pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply conv2 + relu + max pooling
        x = self.pool(F.relu(self.conv3(x)))  # Apply conv3 + relu + max pooling

        x = x.view(-1, 128 * 25 * 25)  # Flatten the output from conv layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Output layer

        return x


import torch.nn.functional as F

class GeneralizedCNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(GeneralizedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        fc_input_size = (input_size[0] // 8) * (input_size[1] // 8) * 128
        self.fc1 = nn.Linear(fc_input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [2]:
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
import os

data_dir = '/kaggle/input/fabric-romo/romo'

# data_dir = "/kaggle/input/pattern-recognition/pattern-recognition"

transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize the image to 150x150
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),          # Convert the image to a tensor
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])
dataset = ImageFolder(data_dir+'/train', transform=transform)
img, label = dataset[0]
print(img.size, label)
print(dataset.classes)


print(len(dataset))

total_size = len(dataset)
val_size = int(0.3 * total_size)
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

batch_size = 16
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, num_workers=4, pin_memory=True)

# model = GeneralizedCNN(input_size=(150, 150), num_classes=18)
model = GeneralizedCNN(input_size=(32, 32), num_classes=18)

device = get_default_device()

train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)


# Initialize the model, loss function, and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop

# Initialize lists to store loss and accuracy
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 100
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_dl:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_dl:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_running_corrects += torch.sum(preds == labels.data)
    
    val_loss = val_running_loss / val_size
    val_acc = val_running_corrects.double() / val_size
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    print(f'Epoch {epoch}/{num_epochs - 1}, '
          f'Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')
print('Finished Training')


# Plot loss and accuracy
epochs = range(num_epochs)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

<built-in method size of Tensor object at 0x79117ba2b740> 0
['abstract', 'animals', 'damask', 'dot_spot', 'floral', 'geometric', 'herringbone_chevron', 'ikat', 'moire', 'motif', 'ombre', 'paisley', 'plaid_check', 'plain_solid', 'semi_plain', 'small_scale', 'stripe', 'trellis']
9630
Epoch 0/99, Train Loss: 2.3850, Train Accuracy: 0.2074, Val Loss: 2.3313, Val Accuracy: 0.2118
Epoch 1/99, Train Loss: 2.2776, Train Accuracy: 0.2308, Val Loss: 2.2094, Val Accuracy: 0.2440
Epoch 2/99, Train Loss: 2.2217, Train Accuracy: 0.2402, Val Loss: 2.2771, Val Accuracy: 0.2260
Epoch 3/99, Train Loss: 2.1573, Train Accuracy: 0.2743, Val Loss: 2.0739, Val Accuracy: 0.3115
Epoch 4/99, Train Loss: 2.0948, Train Accuracy: 0.2980, Val Loss: 2.1019, Val Accuracy: 0.2831
Epoch 5/99, Train Loss: 2.0557, Train Accuracy: 0.3123, Val Loss: 2.0408, Val Accuracy: 0.3250
Epoch 6/99, Train Loss: 2.0342, Train Accuracy: 0.3201, Val Loss: 2.0191, Val Accuracy: 0.3423
Epoch 7/99, Train Loss: 1.9939, Train Accuracy: 0.33

KeyboardInterrupt: 

In [ ]:
# save model
torch.save(model, "/kaggle/working/pr.pth")

In [ ]:
dt = ImageFolder(data_dir+'/train', transform=transform)
dt.classes

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the test data
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize images to 150x150
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load your test dataset (example using CIFAR-10, replace with your dataset)
test_dataset = ImageFolder(data_dir+'/test', transform=transform)
# Define the test data loader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Define the class names (example using CIFAR-10 class names)
class_names = dt.classes

# Function to calculate the accuracy
def calculate_accuracy(model, data_loader, class_names, device):
    model.eval()  # Set the model to evaluation mode
    class_correct = [0] * len(class_names)
    class_total = [0] * len(class_names)
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
    
    overall_accuracy = 100 * total_correct / total_samples
    print(f'Overall Accuracy: {overall_accuracy:.2f}%')
    
    for i in range(len(class_names)):
        if class_total[i] > 0:
            accuracy = 100 * class_correct[i] / class_total[i]
            print(f'Accuracy of {class_names[i]}: {accuracy:.2f}%')
        else:
            print(f'Accuracy of {class_names[i]}: N/A (no samples)')
    
    return overall_accuracy, class_correct, class_total

# Evaluate the model on the test set
calculate_accuracy(model, test_loader, class_names, device)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformations for the image
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize the image to 150x150
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),          # Convert the image to a tensor
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

# Detect the available device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model (assuming it's saved as 'model.pth')
input_size = (150, 150)
num_classes = 18
model = GeneralizedCNN(input_size=input_size, num_classes=num_classes)
model = torch.load('/kaggle/working/pr.pth'))
model.to(device)  # Move the model to the detected device
model.eval()  # Set the model to evaluation mode

# Function to predict the class of an unknown image
def predict_image(image_path, model, transform, class_names, device):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    
    # Preprocess the image
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)  # Move the image to the detected device
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    # Get the predicted class name
    predicted_class = class_names[predicted.item()]
    return predicted_class

# Example usage
image_path = '/kaggle/input/fabric-romo/romo/test/other/7782-01-odette-chamois_01.jpg'  # Replace with the path to your image
class_names = dt.classes  # Replace with your class names if different

predicted_class = predict_image(image_path, model, transform, class_names, device)
print(f'The predicted class for the image is: {predicted_class}')
